### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd
df = pd.read_csv('ml-latest-small/ratings.csv')
df['class'] = df['rating'].apply(lambda x: 'low' if x <= 2 else 'middle' if 2 < x <= 4 else 'high')
df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,middle
1,1,1029,3.0,1260759179,middle
2,1,1061,3.0,1260759182,middle
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,middle


In [2]:
sorted(df.rating.unique())

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [3]:
geo_dat = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
geo_df = pd.read_csv('Python_11_pandas_functions_groupby/keywords.csv')
geo_df.head()

def region_classifier(key):
    key = key.lower()
    for region in geo_dat.keys():
        if key in geo_dat[region]:
            return region
    else:
        return 'undefined'
    
geo_df['region'] = geo_df['keyword'].apply(region_classifier)
geo_df[geo_df['region'] != 'undefined']

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-запад
9084,хабаровск,29655,Дальний Восток
11222,псков,24885,Северо-запад
11785,тула,23898,Центр
20823,петербург,14859,Северо-запад
21445,сахалин,14494,Дальний Восток


### Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
        для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


In [4]:
movies_df = pd.read_csv('ml-25m/movies.csv')

years_str = list(map(str, range(1950, 2010+1)))

def production_year(title):
    for year in years_str:
        if year in title:
            return int(year)
    else:
        return 1900
    
movies_df['year'] = movies_df['title'].apply(production_year)
movies_df
# aggr by year, mean, sort

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
62418,209157,We (2018),Drama,1900
62419,209159,Window of the Soul (2001),Documentary,2001
62420,209163,Bad Poems (2018),Comedy|Drama,1900
62421,209169,A Girl Thing (2001),(no genres listed),2001


In [5]:
rating_df = pd.read_csv('ml-25m/ratings.csv')

In [6]:
joined = pd.merge(movies_df, rating_df, how='right', on='movieId')

In [7]:
pd.set_option('display.max_rows', None)
(joined.
     groupby('year').
     mean()['rating'].
     sort_values(ascending=False)
)

year
1957    3.999492
1954    3.985693
1972    3.956477
1962    3.929445
1952    3.895315
1974    3.891385
1975    3.883071
1964    3.879177
1951    3.876258
1950    3.846525
1966    3.835132
1977    3.829831
1960    3.819289
1959    3.819035
1967    3.815437
1958    3.807918
1971    3.753775
1968    3.749666
1969    3.749487
1979    3.748333
1973    3.744928
1955    3.729484
1976    3.727293
1963    3.722909
1980    3.718424
1953    3.710637
1956    3.707795
1965    3.705861
1961    3.701205
1981    3.691544
1983    3.669378
1970    3.667862
1982    3.604348
1991    3.601201
1900    3.592460
1984    3.578314
2010    3.564217
2006    3.551608
1987    3.540011
1994    3.523614
2009    3.522272
1985    3.519315
1999    3.516408
2008    3.515277
1993    3.512984
2004    3.512853
1988    3.509279
1989    3.507501
2007    3.504555
1986    3.504521
2001    3.492072
2002    3.470507
2003    3.469934
1995    3.462305
1978    3.460018
1992    3.458007
1998    3.455291
1990    3.435736
2005    3

“раньше снимали настоящее кино, не то что сейчас”? 
Это не проверить таким способом, ведь из старых фильмов чаще смотрят высокорейнитговые, а о плохих забывают, тем самым смещая оценку вверх для ранних годов. Нужно усреднять рейтниг сперва внутри фильма, а только затем по годам.

In [8]:
(joined.
    groupby('movieId').mean().
    groupby('year').mean()['rating'].
    sort_values(ascending=False)
)

year
1963.0    3.228859
1961.0    3.208200
1962.0    3.194438
1953.0    3.190671
1950.0    3.179434
1964.0    3.178317
1965.0    3.152148
1952.0    3.145530
1955.0    3.130389
2010.0    3.123972
1954.0    3.122618
1967.0    3.119002
1968.0    3.114046
1969.0    3.111372
1959.0    3.111047
1971.0    3.106319
1983.0    3.099591
1972.0    3.098869
1960.0    3.095334
2003.0    3.093011
1956.0    3.092178
1900.0    3.087217
1973.0    3.082421
2009.0    3.082298
1976.0    3.081141
1958.0    3.080403
1951.0    3.077510
1966.0    3.071588
1993.0    3.071516
1996.0    3.070242
2005.0    3.069747
1975.0    3.068471
1979.0    3.067484
1982.0    3.060996
2008.0    3.060768
2006.0    3.055308
1999.0    3.055008
2004.0    3.053805
1981.0    3.050934
2007.0    3.049558
1957.0    3.048081
1997.0    3.045220
2001.0    3.043944
2002.0    3.041510
1992.0    3.041381
1995.0    3.040016
1970.0    3.037114
1984.0    3.036315
1994.0    3.032198
1977.0    3.025761
2000.0    3.023143
1985.0    3.021609
1978.0 

Теперь 2010 год поднялся из середины в первую десятку, а это последний из наших годов. Поэтому слова “раньше снимали настоящее кино, не то что сейчас” не совсем верны.